# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ROOT_PATH = "/content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%notebook inline

In [3]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 14.9MB/s 
     |████████████████████████████████| 3.0MB 17.0MB/s 
     |████████████████████████████████| 890kB 21.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7a629216cf3c11ab9cde3a68603cfe36c02c88bebd7a6e36f41d360f67c09e19
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import pandas as pd 
import numpy as np
import json

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, XLMRobertaTokenizer, XLMRobertaConfig, DistilBertTokenizer, DistilBertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.6.0+cu101'

In [5]:
def read_json(filename) :
  with open(filename, "r", encoding="utf8") as f:
    obj = json.load(f)
  return obj

def write_json(obj, filename) :
  with open(filename, "w", encoding="utf8") as outfile:
    json.dump(obj, outfile)
  print ("Successfully write JSON obj to", filename)

def write_tsv(obj, filename) :
  file = open(filename, "w", encoding="utf-8") 
  for tokens in obj :
    for token in tokens :
      file.write(token["token"] + "\t" + token["label"] + "\n")
    file.write("==\n\n")
  file.close()
  print ("Successfully write JSON obj to", filename)

def read_tsv(filename) :
  file = open(filename, "r") 
  obj = {}
  items = []
  words = file.read().split("==\n\n")
  sentences = [[tokens.split("\t") for tokens in word.split("\n")] for word in words]
  for tokens in sentences :
    item = []
    for token in tokens :
      if len(token) == 2 :
        tok_lab = {}
        tok_lab["token"] = token[0]
        tok_lab["label"] = token[1]
        item.append(tok_lab)
    if len(item) > 1 :
      items.append(item)
  obj["tokens_labels"] = items
  return obj

def make_unique(obj) :
  unique_list = []
  not_unique_list = 0

  for item in obj :
    if item in unique_list :
      not_unique_list += 1
    else :
      unique_list.append(item)

  obj = unique_list
  print ("Drop", not_unique_list, "Result", len(obj))
  return obj

class FineTunedModel :
  def __init__(self, model_type="mbert", version="1", fine_tuned_type="full", scenario=1, label_type=None) :
    ROOT_PATH = "/content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/"
    if scenario :
      MODEL_PATH = "model/scenario " + str(scenario) + "/" 
    else :
      MODEL_PATH = "model/"
    if not label_type :
      self.__tag_values = ["B-Merek", "I-Merek", "O", "B-NamaProduk", "I-NamaProduk", "B-Varian", "I-Varian", "B-Ukuran", "I-Ukuran", "B-Penggunaan", "I-Penggunaan", "B-Tekstur", "I-Tekstur", "PAD"]
    else :
      self.__tag_values = ["1-B-Merek", "1-I-Merek", "2-O", "3-B-NamaProduk", "3-I-NamaProduk", "4-B-Varian", "4-I-Varian", "5-B-Ukuran", "5-I-Ukuran", "6-B-Penggunaan", "6-I-Penggunaan", "7-B-Tekstur", "7-I-Tekstur", "PAD"]
    self.__model_type = model_type

    # valid_models = ["xlmr", "mbert", "distilbert"]
    valid_models = ["xlmr", "mbert"]
    valid_fine_tuned_type = ["full", "partial"]

    if model_type in valid_models and fine_tuned_type in valid_fine_tuned_type :
      MODEL_FILENAME = model_type + "_v" + version + "_" + fine_tuned_type + ".pth"
      if model_type == "xlmr":
        MODEL_NAME = "xlm-roberta-base"
        self.__tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

      elif model_type == "mbert":
        MODEL_NAME = "bert-base-multilingual-cased"
        self.__tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

      elif model_type == "distilbert" :
        MODEL_NAME = "distilbert-base-multilingual-cased"
        self.__tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

    elif model_type == "xlmr_tokped_ipv_label" :
      MODEL_FILENAME = "model-labelling-tokped-ipv.pth"
      MODEL_NAME = "xlm-roberta-base"
      self.__tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

    elif model_type == "xlmr_tokped_esw_label" :
      MODEL_FILENAME = "model-labelling-tokped-esw.pth"
      MODEL_NAME = "xlm-roberta-base"
      self.__tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)
    
    print ("Load", MODEL_PATH + MODEL_FILENAME)
    self.__model = torch.load(ROOT_PATH + MODEL_PATH + MODEL_FILENAME).cuda()

  def predict_text(self, text) :
    self.__model.eval()
    result = []
    tokenized_sentence = self.__tokenizer.encode(text)
    if (len(tokenized_sentence) > 512) :
      return None
    input_ids = torch.tensor([tokenized_sentence]).cuda()
    with torch.no_grad():
      output = self.__model(input_ids)
    label_indices = np.argmax(output[0].to("cpu").numpy(), axis=2)

    # Join Split Token
    tokens = self.__tokenizer.convert_ids_to_tokens(input_ids.to("cpu").numpy()[0])
    new_tokens, new_labels = [], []
    
    # Initialize Variables for XLM-R Join Token
    result_token = ""
    result_label = None
    
    for token, label_idx in zip(tokens, label_indices[0]):
      if "xlmr" in self.__model_type :
        if token.startswith("▁"):
          if result_token != "" :
            new_labels.append(result_label)
            new_tokens.append(result_token)
          result_token = token[1:]
          result_label = self.__tag_values[label_idx -1]
        else:
          result_token += token
      elif "bert" in self.__model_type :
        if token.startswith("##"):
          new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
          new_labels.append(self.__tag_values[label_idx - 1])
          new_tokens.append(token)
    
    # Append last token from xlmr and remove "</s>" from the token
    if "xlmr" in self.__model_type :
      new_labels.append(result_label)
      new_tokens.append(result_token.rstrip("</s>"))

    for token, label in zip(new_tokens, new_labels):
      elmt = {}
      elmt["token"] = token
      elmt["label"] = label
      result.append(elmt)
    # Remove [CLS] for mbert and distilbert
    del result[0]

    # Remove [SEP] for mbert and distilbert
    if "bert" in self.__model_type : 
      del result[-1]
    return result

  def predict(self, texts) :
    results = {}
    tokens_labels = []
    count = 0
    for idx, text in enumerate(texts) :
      if text != "" :
        result = self.predict_text(text)
        if result != None :
          tokens_labels.append(self.predict_text(text))
        else :
          count += 1
    results["tokens_labels"] = tokens_labels
    print ("Skipped text", count)
    return results

# Environment Check

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [7]:
torch.cuda.get_device_name(0)

'Tesla K80'

# Predict Demo
FineTunedModel(model_type, version, fine_tuned_type, scenario)
Example : FineTunedModel("bert", 1, "full", 1)

In [ ]:
# fine_tuned_model = FineTunedModel("mbert", 1, "full", 1)

In [ ]:
# fine_tuned_model.predict_text("Inez Precious Powdery Cake - Beige New Case [SEP] Buat wajahmu tampak lebih cantik dengan Inez Precious Powdery Cake-Beige 10. Hadir dengan kombinasi ekstrak mulbery, vitamin A palmitat dan E acetat, serta anti UV yang bantu lindungi kulit wajah selama beraktivitas. DETAIL Mecerahkan Kulit Wajah Hadir dengan formulasi ekstrak mulbery, Inez Precious Powdery Cake bantu cerahkan kulit wajahmu seharian. Menjaga Kelembaban Vitamin A palmitat dan E acetat yang terkandung dalam Inez Precious Powdery Cake dapat membuat kulit wajahmu tetap lembab di setiap kesempatan. Anti Oksidan Tidak hanya melembabkan kulit wajah, Inez Precious Powdery Cake mengandung vitamin A palmitat dan E acetat juga berfungsi sebagai anti oksidan bagi kulit wajahmu. Anti UV Diformulasikan dengan kandungan anti UV, Inez Precious Powdery Cake dapat melindungi wajahmu meski harus beraktivitas di luar ruangan seharian. Menutupi Noda Hitam Gak perlu takut lagi dengan noda hitam di wajahmu. Kini ada Inez Precious Powdery Cake yang dapat membantu tutupi noda hitam pada wajah dan buat penampilanmu lebih sempurna. Halal Inez Precious Powdery Cake sangat aman dan nyaman ketika digunakan karena sudah memiliki lisensi halal dari MUI Indonesia.")

In [ ]:
# fine_tuned_model.predict_text("Emina Double Agent Eyebrow 0.25 g [SEP] Retractable Eye brow pencil yang dilengkapi dengan sikat lembut 1. Bisa menggambar alis dengan mudah karena memiliki ujung yang bersudut 2. Ujung yang satu berupa pensil dengan formula lembut untuk membentuk alis yang terlihat alami 3. Ujung satunya berupa sikat lembut berbentuk ulir untuk hasil alis sempurna 4. Tidak perlu diserut (cukup diputar) 5. Tersedia dalam 2 varian (Brown dan Black) Agent Eyebrow Black 0.25 g NA11151202733 Agent Eyebrow Brown 0.25 g NA11171208384")

In [ ]:
# fine_tuned_model = FineTunedModel("mbert", 1, "full", 2)

In [ ]:
# fine_tuned_model.predict_text("Inez Precious Powdery Cake - Beige New Case [SEP] Buat wajahmu tampak lebih cantik dengan Inez Precious Powdery Cake-Beige 10. Hadir dengan kombinasi ekstrak mulbery, vitamin A palmitat dan E acetat, serta anti UV yang bantu lindungi kulit wajah selama beraktivitas. DETAIL Mecerahkan Kulit Wajah Hadir dengan formulasi ekstrak mulbery, Inez Precious Powdery Cake bantu cerahkan kulit wajahmu seharian. Menjaga Kelembaban Vitamin A palmitat dan E acetat yang terkandung dalam Inez Precious Powdery Cake dapat membuat kulit wajahmu tetap lembab di setiap kesempatan. Anti Oksidan Tidak hanya melembabkan kulit wajah, Inez Precious Powdery Cake mengandung vitamin A palmitat dan E acetat juga berfungsi sebagai anti oksidan bagi kulit wajahmu. Anti UV Diformulasikan dengan kandungan anti UV, Inez Precious Powdery Cake dapat melindungi wajahmu meski harus beraktivitas di luar ruangan seharian. Menutupi Noda Hitam Gak perlu takut lagi dengan noda hitam di wajahmu. Kini ada Inez Precious Powdery Cake yang dapat membantu tutupi noda hitam pada wajah dan buat penampilanmu lebih sempurna. Halal Inez Precious Powdery Cake sangat aman dan nyaman ketika digunakan karena sudah memiliki lisensi halal dari MUI Indonesia.")

In [ ]:
# fine_tuned_model.predict_text("Emina Double Agent Eyebrow 0.25 g [SEP] Retractable Eye brow pencil yang dilengkapi dengan sikat lembut 1. Bisa menggambar alis dengan mudah karena memiliki ujung yang bersudut 2. Ujung yang satu berupa pensil dengan formula lembut untuk membentuk alis yang terlihat alami 3. Ujung satunya berupa sikat lembut berbentuk ulir untuk hasil alis sempurna 4. Tidak perlu diserut (cukup diputar) 5. Tersedia dalam 2 varian (Brown dan Black) Agent Eyebrow Black 0.25 g NA11151202733 Agent Eyebrow Brown 0.25 g NA11171208384")

In [8]:
fine_tuned_model = FineTunedModel("xlmr", "final", "full", 3)


Load model/scenario 3/xlmr_vfinal_full.pth


In [9]:
fine_tuned_model.predict_text("Inez Precious Powdery Cake - Beige New Case [SEP] Buat wajahmu tampak lebih cantik dengan Inez Precious Powdery Cake-Beige 10. Hadir dengan kombinasi ekstrak mulbery, vitamin A palmitat dan E acetat, serta anti UV yang bantu lindungi kulit wajah selama beraktivitas. DETAIL Mecerahkan Kulit Wajah Hadir dengan formulasi ekstrak mulbery, Inez Precious Powdery Cake bantu cerahkan kulit wajahmu seharian. Menjaga Kelembaban Vitamin A palmitat dan E acetat yang terkandung dalam Inez Precious Powdery Cake dapat membuat kulit wajahmu tetap lembab di setiap kesempatan. Anti Oksidan Tidak hanya melembabkan kulit wajah, Inez Precious Powdery Cake mengandung vitamin A palmitat dan E acetat juga berfungsi sebagai anti oksidan bagi kulit wajahmu. Anti UV Diformulasikan dengan kandungan anti UV, Inez Precious Powdery Cake dapat melindungi wajahmu meski harus beraktivitas di luar ruangan seharian. Menutupi Noda Hitam Gak perlu takut lagi dengan noda hitam di wajahmu. Kini ada Inez Precious Powdery Cake yang dapat membantu tutupi noda hitam pada wajah dan buat penampilanmu lebih sempurna. Halal Inez Precious Powdery Cake sangat aman dan nyaman ketika digunakan karena sudah memiliki lisensi halal dari MUI Indonesia.")

[{'label': 'B-Merek', 'token': 'Inez'},
 {'label': 'B-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake'},
 {'label': 'O', 'token': '-'},
 {'label': 'B-Varian', 'token': 'Beige'},
 {'label': 'O', 'token': 'New'},
 {'label': 'O', 'token': 'Case'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Buat'},
 {'label': 'B-Penggunaan', 'token': 'wajahmu'},
 {'label': 'O', 'token': 'tampak'},
 {'label': 'O', 'token': 'lebih'},
 {'label': 'O', 'token': 'cantik'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'B-Merek', 'token': 'Inez'},
 {'label': 'B-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake-Beige'},
 {'label': 'O', 'token': '10.'},
 {'label': 'O', 'token': 'Hadir'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'kombinasi'},
 {'label': 'O', 'token': 'ekstrak'},
 {'label': 'O', 'token': 'mulbery,'},
 {'label': '

In [11]:
fine_tuned_model.predict_text("Sariayu Tinted Moisturizer [SEP] Pelembap berwarna dengan tekstur ringan , mengandung Ekstrak Bunga Gardenia dan Vitamin E sebagai antioksidan , diperkaya dengan natural moisturizer untuk menjaga kelembapan kulit dan juga SPF 20 yang melindungi kulit dari pengaruh buruk sinar matahari . Sariayu Gold Tinted Moisturizer , memberikan warna kulit lebih merata dan tampak alami . Tersedia dalam tiga pilihan warna : 01 Light 02 Natural 03 Dark Isi 30 ml")

[{'label': 'B-Merek', 'token': 'Sariayu'},
 {'label': 'B-NamaProduk', 'token': 'Tinted'},
 {'label': 'I-NamaProduk', 'token': 'Moisturizer'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Pelembap'},
 {'label': 'O', 'token': 'berwarna'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'tekstur'},
 {'label': 'O', 'token': 'ringan'},
 {'label': 'O', 'token': ','},
 {'label': 'O', 'token': 'mengandung'},
 {'label': 'O', 'token': 'Ekstrak'},
 {'label': 'O', 'token': 'Bunga'},
 {'label': 'O', 'token': 'Gardenia'},
 {'label': 'O', 'token': 'dan'},
 {'label': 'O', 'token': 'Vitamin'},
 {'label': 'O', 'token': 'E'},
 {'label': 'O', 'token': 'sebagai'},
 {'label': 'O', 'token': 'antioksidan'},
 {'label': 'O', 'token': ','},
 {'label': 'O', 'token': 'diperkaya'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'natural'},
 {'label': 'O', 'token': 'moisturizer'},
 {'label': 'O', 'token': 'untuk'},
 {'label': 'O', 'token': 'menjaga'},
 {'label': 'O', 'token

In [ ]:
# fine_tuned_model.predict_text("Emina Double Agent Eyebrow 0.25 g [SEP] Retractable Eye brow pencil yang dilengkapi dengan sikat lembut 1. Bisa menggambar alis dengan mudah karena memiliki ujung yang bersudut 2. Ujung yang satu berupa pensil dengan formula lembut untuk membentuk alis yang terlihat alami 3. Ujung satunya berupa sikat lembut berbentuk ulir untuk hasil alis sempurna 4. Tidak perlu diserut (cukup diputar) 5. Tersedia dalam 2 varian (Brown dan Black) Agent Eyebrow Black 0.25 g NA11151202733 Agent Eyebrow Brown 0.25 g NA11171208384")

In [ ]:
# fine_tuned_model = FineTunedModel("xlmr", 1, "full", 2)

In [ ]:
# fine_tuned_model.predict_text("Inez Precious Powdery Cake - Beige New Case [SEP] Buat wajahmu tampak lebih cantik dengan Inez Precious Powdery Cake-Beige 10. Hadir dengan kombinasi ekstrak mulbery, vitamin A palmitat dan E acetat, serta anti UV yang bantu lindungi kulit wajah selama beraktivitas. DETAIL Mecerahkan Kulit Wajah Hadir dengan formulasi ekstrak mulbery, Inez Precious Powdery Cake bantu cerahkan kulit wajahmu seharian. Menjaga Kelembaban Vitamin A palmitat dan E acetat yang terkandung dalam Inez Precious Powdery Cake dapat membuat kulit wajahmu tetap lembab di setiap kesempatan. Anti Oksidan Tidak hanya melembabkan kulit wajah, Inez Precious Powdery Cake mengandung vitamin A palmitat dan E acetat juga berfungsi sebagai anti oksidan bagi kulit wajahmu. Anti UV Diformulasikan dengan kandungan anti UV, Inez Precious Powdery Cake dapat melindungi wajahmu meski harus beraktivitas di luar ruangan seharian. Menutupi Noda Hitam Gak perlu takut lagi dengan noda hitam di wajahmu. Kini ada Inez Precious Powdery Cake yang dapat membantu tutupi noda hitam pada wajah dan buat penampilanmu lebih sempurna. Halal Inez Precious Powdery Cake sangat aman dan nyaman ketika digunakan karena sudah memiliki lisensi halal dari MUI Indonesia.")

In [ ]:
# fine_tuned_model.predict_text("Emina Double Agent Eyebrow 0.25 g [SEP] Retractable Eye brow pencil yang dilengkapi dengan sikat lembut 1. Bisa menggambar alis dengan mudah karena memiliki ujung yang bersudut 2. Ujung yang satu berupa pensil dengan formula lembut untuk membentuk alis yang terlihat alami 3. Ujung satunya berupa sikat lembut berbentuk ulir untuk hasil alis sempurna 4. Tidak perlu diserut (cukup diputar) 5. Tersedia dalam 2 varian (Brown dan Black) Agent Eyebrow Black 0.25 g NA11151202733 Agent Eyebrow Brown 0.25 g NA11171208384")

# Tokped IPV

In [ ]:
# fine_tuned_model_ipv = FineTunedModel("xlmr_tokped_ipv_label", None, None, None, 1)

Load model/model-labelling-tokped-ipv.pth


In [ ]:
# fine_tuned_model_ipv.predict_text("Inez Compact Powder 09 Ivory Glow Bedak Padat [SEP] * WAJIB BELI BUBBLE WRAP * 09 IVORY GLOW")

[{'label': '1-B-Merek', 'token': 'Inez'},
 {'label': '3-B-NamaProduk', 'token': 'Compact'},
 {'label': '3-I-NamaProduk', 'token': 'Powder'},
 {'label': '4-B-Varian', 'token': '09'},
 {'label': '4-I-Varian', 'token': 'Ivory'},
 {'label': '4-I-Varian', 'token': 'Glow'},
 {'label': '2-O', 'token': 'Bedak'},
 {'label': '2-O', 'token': 'Padat'},
 {'label': '2-O', 'token': '[SEP]'},
 {'label': '2-O', 'token': '*'},
 {'label': '2-O', 'token': 'WAJIB'},
 {'label': '2-O', 'token': 'BELI'},
 {'label': '2-O', 'token': 'BUBBLE'},
 {'label': '2-O', 'token': 'WRAP'},
 {'label': '2-O', 'token': '*'},
 {'label': '4-B-Varian', 'token': '09'},
 {'label': '4-I-Varian', 'token': 'IVORY'},
 {'label': '4-I-Varian', 'token': 'GLOW'}]

In [ ]:
# tokped_ipv_filename = ROOT_PATH + "dataset/tokped_ipv.txt"
# texts = open(tokped_ipv_filename, "r").readlines()
# texts = make_unique(texts)
# print(len(texts))

Drop 222 Result 1198
1198


In [ ]:
# result = fine_tuned_model_ipv.predict(texts)
# len(result['tokens_labels'])

Skipped text 0


1198

In [ ]:
# write_tsv(result["tokens_labels"], ROOT_PATH + "dataset/tokped_ipv-label.txt")

Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/tokped_ipv-label.txt


# Tokped ESW

In [ ]:
# fine_tuned_model_esw = FineTunedModel("xlmr_tokped_esw_label", None, None, None, 1)

# tokped_esw_filename = ROOT_PATH + "dataset/tokped_esw.txt"
# texts = open(tokped_esw_filename, "r").readlines()
# texts = make_unique(texts)
# print(len(texts))

# result = fine_tuned_model_esw.predict(texts)
# len(result['tokens_labels'])

# write_tsv(result["tokens_labels"], ROOT_PATH + "dataset/tokped_esw-label.txt")

Load model/model-labelling-tokped-esw.pth
Drop 117 Result 1161
1161
Skipped text 0
Successfully write JSON obj to /content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/dataset/tokped_esw-label.txt


# Analysis Preparation

In [ ]:
# labels = []
# for lab in testing_labels :
#   for l in lab :
#     labels.append(l)

# print (set(labels))

In [ ]:
# model_type = "xlmr"
# FINE_TUNING_TYPE = "full"
# VERSION = 2
# SCENARIO = 1
# RESULT_INFO = model_type + "_v" + str(VERSION) + "_" + FINE_TUNING_TYPE
# MODEL_FILENAME = RESULT_INFO  + ".pth"
# print (MODEL_FILENAME)

# ANALYZE_PATH = ROOT_PATH + "analyze/scenario " + str(SCENARIO) + + "/" + RESULT_INFO + ".txt"
# DATASET_PATH = ROOT_PATH + "dataset/version " + str(VERSION) + '/scenario-' + str(SCENARIO) + "-test.txt"

In [ ]:
# texts = open(DATASET_PATH, "r").readlines()
# print(texts[1])

In [ ]:
# fine_tuned_model = FineTunedModel(model_type, VERSION, FINE_TUNING_TYPE, SCENARIO)
# results = fine_tuned_model.predict(texts)

In [ ]:
# file = open(ANALYZE_PATH, "w", encoding="utf-8") 
# for tokens in results :
#   for token in tokens :
#     file.write(token['token'] + "\t" + token['label'] + "\n")
#   file.write("==\n\n")
# file.close()

# Tokopedia Automatic Labelling

In [ ]:
# texts = open(ROOT_PATH + "dataset/tokopedia.txt", "r").readlines()
# print(texts[1])

In [ ]:
# print (predict_text(texts[0]))

In [ ]:
# obj = {}
# tokens_labels = []
# count = 0

# for idx, text in enumerate(texts) :
#   result = predict_text(text)
#   if result != None :
#     tokens_labels.append(predict_text(text))
#   else :
#     count += 1

# obj['tokens_labels'] = tokens_labels
# print ('Skipped text', count)

In [ ]:
# print (len(obj['tokens_labels']))

In [ ]:
# writeJSON(obj, ROOT_PATH + "dataset/tokopedia.json")